In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import one_hot
import tensorflow.keras.utils as ku 
import numpy as np
import pandas as pd

In [2]:
data = open('test1.txt',encoding="utf8").read()
data

'Fishy Business\nMan\'s got bigger fish \nTo fry,\nLeaving the smaller fish \nTo Die,\nStill always wondering\nSadness, why.\nIf livin\'s easy,\nDying\'s an escape.\nIf being normal is easy,\nPut on the cape.\nIf being fat is easy,\nShed all the shape.\nIf speaking is easy,\nLay down the tape.\n\nThe One Who Was Meant To Be-\n\nWhen I became we\nNothing seemed difficult \nThe world was indeed made of rainbows\nOf unicorns and extra crispy french fries\nYou made me complete\nYet somehow the darkness never left\nFor the wounds had never healed\nGrey hid behind the curtain of colors\nWanting to crush the horn of hope\nI wanted to be me again\nFor the inverted M was never meant to be.\nYou took my heart and put it right\nIt was but my mind dwelling on the sorrows of past\nMy broken teeth, my swollen soul\nFight had already boarded the flight\nThe one overbooked with my emotions\nIt was never supposed to fly again\nYet you were the pilot of my dreams\nMy defence against destiny\nI hope to b

In [3]:
corpus = data.lower().split("\n")
vocab=20000
token = Tokenizer(num_words=vocab,filters='')
token.fit_on_texts(corpus)
input_seq = []
print(token)
total = len(token.word_index)+1

In [4]:
for line in corpus:
	seq = token.texts_to_sequences([line])[0]
	for i in range(1, len(seq)):
		n_gram_seq = seq[:i+1]
		input_seq.append(n_gram_seq)
input_seq

[[490, 282],
 [491, 107],
 [491, 107, 194],
 [491, 107, 194, 283],
 [2, 492],
 [284, 1],
 [284, 1, 493],
 [284, 1, 493, 283],
 [2, 494],
 [94, 85],
 [94, 85, 495],
 [496, 497],
 [47, 498],
 [47, 498, 77],
 [499, 48],
 [499, 48, 500],
 [47, 108],
 [47, 108, 501],
 [47, 108, 501, 10],
 [47, 108, 501, 10, 77],
 [195, 13],
 [195, 13, 1],
 [195, 13, 1, 502],
 [47, 108],
 [47, 108, 503],
 [47, 108, 503, 10],
 [47, 108, 503, 10, 77],
 [504, 26],
 [504, 26, 1],
 [504, 26, 1, 505],
 [47, 506],
 [47, 506, 10],
 [47, 506, 10, 77],
 [196, 109],
 [196, 109, 1],
 [196, 109, 1, 507],
 [1, 17],
 [1, 17, 40],
 [1, 17, 40, 14],
 [1, 17, 40, 14, 95],
 [1, 17, 40, 14, 95, 2],
 [1, 17, 40, 14, 95, 2, 508],
 [23, 6],
 [23, 6, 138],
 [23, 6, 138, 8],
 [139, 285],
 [139, 285, 140],
 [1, 32],
 [1, 32, 14],
 [1, 32, 14, 509],
 [1, 32, 14, 509, 49],
 [1, 32, 14, 509, 49, 5],
 [1, 32, 14, 509, 49, 5, 510],
 [5, 511],
 [5, 511, 7],
 [5, 511, 7, 512],
 [5, 511, 7, 512, 513],
 [5, 511, 7, 512, 513, 514],
 [5, 511, 7

In [8]:
max_seq_len = max([len(x) for x in input_seq])
input_seq = np.array(pad_sequences(input_seq, maxlen=max_seq_len, padding='pre'))
input_seq

array([[   0,    0,    0, ...,    0,  490,  282],
       [   0,    0,    0, ...,    0,  491,  107],
       [   0,    0,    0, ...,  491,  107,  194],
       ...,
       [   0,    0,    0, ...,  317,    2, 1892],
       [   0,    0,    0, ...,    2, 1892,    3],
       [   0,    0,    0, ..., 1892,    3,  488]], dtype=int32)

In [9]:
predictor = input_seq[:,:-1]
label = input_seq[:,-1]
label = ku.to_categorical(label,num_classes=total,dtype = "int32")

In [10]:
model = Sequential()
model.add(Embedding(total, 256, input_length=max_seq_len-1))
model.add(Bidirectional(LSTM(256, return_sequences = True)))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(total/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 11, 256)           484608    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 11, 512)           1050624   
_________________________________________________________________
dropout_1 (Dropout)          (None, 11, 512)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_2 (Dense)              (None, 946)               243122    
_________________________________________________________________
dense_3 (Dense)              (None, 1893)              1792671   
Total params: 4,358,481
Trainable params: 4,358,481
Non-trainable params: 0
____________________________________________

In [11]:
apply = model.fit(predictor, label, epochs=100, verbose=1)

Epoch 1/100
121/121 [==============================] - 34s 280ms/step - loss: 7.8945 - accuracy: 0.0522
Epoch 2/100
121/121 [==============================] - 34s 278ms/step - loss: 6.4838 - accuracy: 0.0522
Epoch 3/100
121/121 [==============================] - 34s 279ms/step - loss: 6.2202 - accuracy: 0.0584
Epoch 4/100
121/121 [==============================] - 35s 288ms/step - loss: 6.0385 - accuracy: 0.0530
Epoch 5/100
121/121 [==============================] - 34s 284ms/step - loss: 5.9341 - accuracy: 0.0569
Epoch 6/100
121/121 [==============================] - 37s 304ms/step - loss: 5.8216 - accuracy: 0.0610
Epoch 7/100
121/121 [==============================] - 34s 281ms/step - loss: 5.7329 - accuracy: 0.0701
Epoch 8/100
121/121 [==============================] - 33s 276ms/step - loss: 5.6478 - accuracy: 0.0801
Epoch 9/100
121/121 [==============================] - 30s 247ms/step - loss: 5.5663 - accuracy: 0.0825
Epoch 10/100
121/121 [==============================] - 28s 232m

121/121 [==============================] - 28s 230ms/step - loss: 1.8264 - accuracy: 0.6520
Epoch 80/100
121/121 [==============================] - 27s 227ms/step - loss: 1.8003 - accuracy: 0.6634
Epoch 81/100
121/121 [==============================] - 28s 235ms/step - loss: 1.7540 - accuracy: 0.6774
Epoch 82/100
121/121 [==============================] - 29s 236ms/step - loss: 1.7321 - accuracy: 0.6776
Epoch 83/100
121/121 [==============================] - 30s 245ms/step - loss: 1.6992 - accuracy: 0.6836
Epoch 84/100
121/121 [==============================] - 30s 244ms/step - loss: 1.6701 - accuracy: 0.6983
Epoch 85/100
121/121 [==============================] - 29s 238ms/step - loss: 1.6501 - accuracy: 0.6973
Epoch 86/100
121/121 [==============================] - 28s 235ms/step - loss: 1.6281 - accuracy: 0.6980
Epoch 87/100
121/121 [==============================] - 28s 228ms/step - loss: 1.6091 - accuracy: 0.7048
Epoch 88/100
121/121 [==============================] - 27s 225ms/st

In [12]:
model.save('success1.h5')

In [14]:
seed_text = "A sunny day"
next_words = 50
  
for _ in range(next_words):
	seq = token.texts_to_sequences([seed_text])[0]
	seq = pad_sequences([seq], maxlen=max_seq_len-1, padding='pre')
	predicted = model.predict_classes(seq, verbose=0)
	output_word = ""
	for word, index in token.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

A sunny day with turtle in the way, to be given; a name to sail. added, scratches on your dry skin, hay meant and music and take worth (lol) easy, behind the tails in which once i tie you again in a movie, and day day through their fingers tight then came the
